In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a request to the website
url = 'https://www.airlinequality.com/airline-reviews/british-airways'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the reviews using the itemprop attribute
reviews = []
review_elements = soup.find_all(itemprop='review')

for element in review_elements:
    review = element.get_text(strip=True)
    reviews.append(review)

# Store the reviews in a DataFrame
df = pd.DataFrame(reviews, columns=['review'])
print(df.head())


                                              review
0  9/10"Very good flight"Guy Senior(United Kingdo...
1  7/10"relatively comfortable elderly plane"Simo...
2  1/10"70 days chasing BA’s complaints departmen...
3  1/10"BA refused to reimburse me"Michael Chasta...
4  1/10"the flight was delayed"S Herron(Netherlan...


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a request to the website
url = 'https://www.airlinequality.com/airline-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=100'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')


In [10]:
reviews = []
statuses = []

# Find all review elements
review_elements = soup.find_all('div', class_='text_content')  # Replace 'text_content' with actual class name

for element in review_elements:
    full_text = element.get_text(strip=True)
    if "✅Trip Verified|" in full_text:
        status = "Verified"
        review = full_text.replace("✅Trip Verified|", "").strip()
    elif "Not Verified|" in full_text:
        status = "Not Verified"
        review = full_text.replace("Not Verified|", "").strip()
    else:
        status = "Unknown"
        review = full_text.strip()

    reviews.append(review)
    statuses.append(status)

# Store the reviews and statuses in a DataFrame
df = pd.DataFrame({
    'review': reviews,
    'status': statuses
})
print(df)


                                               review        status
0   Very good flight following an equally good fli...  Not Verified
1   An hour's delay due to late arrival of the inc...  Not Verified
2   I booked through BA because Loganair don’t hav...      Verified
3   British airways lost bags in LHR then found th...      Verified
4   The check in process and reward/loyalty progra...      Verified
..                                                ...           ...
95  British Airways new Club Suite is a marked imp...      Verified
96  Four very pleasant, on time flights with frien...      Verified
97  We travel extensively over 25 years and tend t...  Not Verified
98  Flight delayed an hour due to bad weather arou...      Verified
99  A very full flight made Premium Economy feel a...      Verified

[100 rows x 2 columns]


In [11]:
len(df)

100

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_all_pages(base_url, start_page, end_page):
    all_reviews = []
    all_statuses = []

    for i in range(start_page, end_page + 1):
        url = f'{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize=100'
        response = requests.get(url)

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all review elements
        review_elements = soup.find_all('div', class_='text_content')  

        for element in review_elements:
            full_text = element.get_text(strip=True)
            if "✅Trip Verified|" in full_text:
                status = "Verified"
                review = full_text.replace("✅Trip Verified|", "").strip()
            elif "Not Verified|" in full_text:
                status = "Not Verified"
                review = full_text.replace("Not Verified|", "").strip()
            else:
                status = "Unknown"
                review = full_text.strip()

            all_reviews.append(review)
            all_statuses.append(status)

    # Store the reviews and statuses in a DataFrame
    df = pd.DataFrame({
        'review': all_reviews,
        'status': all_statuses
    })
    
    return df

# Define the base URL and page range
base_url = 'https://www.airlinequality.com/airline-reviews/british-airways'
start_page = 1
end_page = 40  

# Scrape all pages and get the DataFrame
reviews_df = scrape_all_pages(base_url, start_page, end_page)
print(reviews_df)


                                                 review        status
0     Very good flight following an equally good fli...  Not Verified
1     An hour's delay due to late arrival of the inc...  Not Verified
2     I booked through BA because Loganair don’t hav...      Verified
3     British airways lost bags in LHR then found th...      Verified
4     The check in process and reward/loyalty progra...      Verified
...                                                 ...           ...
3906  Flew LHR - VIE return operated by bmi but BA a...       Unknown
3907  LHR to HAM. Purser addresses all club passenge...       Unknown
3908  My son who had worked for British Airways urge...       Unknown
3909  London City-New York JFK via Shannon on A318 b...       Unknown
3910  SIN-LHR BA12 B747-436 First Class. Old aircraf...       Unknown

[3911 rows x 2 columns]


In [5]:
def scrape_review_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    reviews = []

    review_elements = soup.find_all('article', itemprop='review')

    for element in review_elements:
        review_data = {}

        review_data['verification_status'] = "Verified" if "✅Trip Verified|" in element.get_text() else "Not Verified"
        review_body = element.find('div', class_='text_content').get_text(strip=True).replace("✅Trip Verified|", "").replace("Not Verified|", "").strip()
        review_data['review_body'] = review_body

        rows = element.find_all('tr')
        for row in rows:
            header = row.find('td', class_='review-rating-header')
            value = row.find('td', class_='review-value')
            if header and value:
                review_data[header.get_text(strip=True)] = value.get_text(strip=True)

        reviews.append(review_data)

    df = pd.DataFrame(reviews)
    return df


In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)
# Define the URL
url = 'https://www.airlinequality.com/airline-reviews/british-airways'

# Scrape the reviews
reviews_df = scrape_review_details(url)
print(reviews_df)

  verification_status                                        review_body  \
0        Not Verified  Very good flight following an equally good fli...   
1        Not Verified  An hour's delay due to late arrival of the inc...   
2        Not Verified  I booked through BA because Loganair don’t hav...   
3        Not Verified  British airways lost bags in LHR then found th...   
4        Not Verified  The check in process and reward/loyalty progra...   
5        Not Verified  We flew in November 2023, but it took this lon...   
6        Not Verified  I left for London from Johannesburg at 21:15 o...   
7        Not Verified  After an excellent flight on a 777 CPT to LHR ...   
8        Not Verified  On a recent flight from Cyprus BA621 on 23/11/...   
9        Not Verified  Flight BA 0560 arrived in Rome on 11 December ...   

  Aircraft Type Of Traveller        Seat Type  \
0     A320      Solo Leisure   Business Class   
1     A319    Family Leisure    Economy Class   
2      NaN      